# Neural Networks II

In [ ]:
%pip install keras_nlp
%pip install tensorflow_datasets
%pip install transformers
%pip install tensorflow-hub

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from transformers import pipeline, AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, classification_report, balanced_accuracy_score

## Neural Networks for Text Data

Neural networks are extremely flexible, which allows you to use them for all kinds of data. We've already seen this with data that was in a 2-dimensional format with images. They can also be used for text data to do tasks such as sentiment analysis using supervised learning.



Go ahead and run this (it will take a moment to finish) and we'll talk about it in a moment:

In [ ]:
url = "https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2"
# using the universal sentence encoder
embed_layer_wrapper = hub.KerasLayer(url, 
                                     trainable=False, 
                                     dtype=tf.string)

# Reviews

We'll start by reloading the IMDB movie review corpus that we used a couple of weeks ago. Just to refresh your memory: this is a subset from a larger corpus of user generated IMDB reviews. The dataset contains the full text of each review, along with a numeric label that is equal to 0 if the review was negative and 1 if the review is positive. Because this is just an example data set, there's actually an even split between positive and negative reviews here, so we have a more-or-less balanced sample of 2500 positive reviews and 2500 negative reviews to work with:


In [ ]:
reviews = pd.read_csv("imdb_reviews.csv")
reviews.head()

Just as in previous classes, we're going to be evalauting a model here by creating separate training and testing datasets. We'll also convert these datasets to tensors in order to make it easier to work with them in tensorflow

In [ ]:


train_examples,  test_examples= train_test_split(reviews,
                                     test_size=0.20, # 20% of observations for validation
                                     random_state = 999) # this is a random process, so you want to set a random seed! 


# convert to tensor objects
train_tensor = tf.convert_to_tensor(train_examples['text'])
test_tensor = tf.convert_to_tensor(test_examples['text'])
train_labels = tf.convert_to_tensor(train_examples['label'])
test_labels = tf.convert_to_tensor(test_examples['label'])

In [ ]:
print(f"Training entries: {len(train_examples)}, test entries: {len(test_examples)}")

## Embeddings and Transformers

In a previous class, we trained a naive bayes classifier to distinguish positive from negative IMDB reviews with a fairly high degree (~84%) accuracy. 

Now, we're going to try to do the same task using a neural network trained on a sentence embedding model. **Text Embeddings** represent one way that analysts can move away from the bag-of-words model to create classifiers that can account for things like word order, synonyms and antonyms and complex grammatical relationships.

Word embedding models can take strings of text and convert them into a "dense" vector of numbers whose values reflect some kind of abstract meaning. The precise method for creating them will be different depending on the model, but a typical approach is to use some text as training data and then "predict" some randomly held out terms or context. Words that have similar meanings will end up with similar weights in the model, because they'll occur in similar contexts.

In a well-trained word-embedding model, words with similar meanings will have similar values (<a href='https://projector.tensorflow.org/'>there's a good visual representation here</a>). Instead of using a bag-of-words as our input for a classifier, we can pass our text through an embedding model to get a representation that can account for things like synonyms and context.

A newer approach, called a transformer, goes a step further and trains a model that can encode both semantic meaning and information about the context in which a word occurs, which means we no longer need to rely on the bag-of-words assumption when working with text. Transformers can take a full sentence and spit out a series of numbers that can function as a sort of numeric representation of meaning.



The `embed` object we downloaded earlier is a pre-trained transformer model that is built for general-purpose applications. It takes a list of strings as inputs and returns a vector of 512 numbers that represent that sentence's "location" in a 512 dimension space. Here's an example of embedding a sentence and then getting the first 10 elements of the embedding vector:

In [ ]:
# embedding a sentence about catci and looking at the first 10 elements

embed(["The rattail cactus is native to Mexico."])[0][:10]

Note that we don't really need to do much (or any) pre-processing of the texts in order for this to give good results. The model is trained on mostly unprocessed text, so things like stripping punctuation or removing stopwords often does more harm than good.

To illustrate what why the embedding is useful, we can use a little code from the <a href='https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder'>online documentation</a> that will allow us to visualize the similarities between the embeddings produced by different sentences:

In [ ]:
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  sns.set(font_scale=1)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")

def run_and_plot(messages_):
  message_embeddings_ = embed(messages_)
  plot_similarity(messages_, message_embeddings_, 90)

Below are some sentences from different wikipedia entries. The first two are from the entry on *Citizen Kane*, the last two are from entries on cacti. Note that the terms in both groups share very few terms overall, but take a look at their similarities as measured by the inner products of their respective embeddings:

In [ ]:
run_and_plot([
    # two sentences from the Wikipedia entry for citizen kane
    "Citizen Kane is often cited as the greatest film ever.",
    "Hollywood had shown interest in Welles in 1936.",
    # sentences from entries on cacti
    "The rattail cactus is native to Mexico.",
    "Prickly pears are frequently found around California."])


In essence, text embeddings give us a more flexibile way to represent text that can account for nuanced aspects of meaning and context, so that sentences about the same general idea are "close" in the embedding space even if they share none of the exact same terms. Feeding these inputs - instead of a simple bag of words - into a machine learning model, can allow us to make more effective use of the same data.

## Fitting the model

Now, let's fit a model to predict movie reviews that uses the embedding model. We'll use the embedding layer as our input layer and then include two hidden layers and a sigmoid output layer that will return our predicted probability of a review being negative or positive

In [ ]:
model = Sequential([
   
    embed_layer_wrapper,
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
    ])



After that, we compile our model and then train it for 15 epochs. 

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


history = model.fit(train_tensor, 
                    train_labels,
                    epochs=15,
                    batch_size=500,
                    validation_data=(test_tensor, 
                                     test_labels),
                    verbose=1
                   )

Now we can generate some predictions and look at our results:

In [ ]:
# generate predictions from test data
preds = model.predict(test_tensor).flatten()>=.5



<h2 style="color:red;font-weight:bold">Question 1 </h2>

<span style="color:red;font-weight:bold">Create a confusion matrix and classification report from the predictions and assess the quality of the classifier</span>

How does this do? Does it outperform the naive bayes classifier? Why might this be? 

## Changes to the Model

We can make changes to the model to add more layers, use more nodes, train it for longer, or even use a different kind of model. This is part of the overall process for finding the model that has the best performance in terms of accuracy. In reality, we would do these steps many, many times, tuning our model so that it is as good as possible.


<h2 style="color:red;font-weight:bold">Question 2 </h2>

<span style="color:red;font-weight:bold">Change something about the model above and compare your results.</span>

(Some options are: add an additional hidden layer, run the same model for more epochs, add more nodes to one or more of the layers, or add <a href='https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout'>dropout</a>)

### Pre-built models from Hugging Face

In reality, the full IMDB reviews corpus is much larger than what we've been using here, so we would also want to use that data in its entirety for a real world application, but since that takes a while to train, we can use a pre-made model that was trained on this data set to get a sense of how well we could do if we did some more fine-tuning.

The [Hugging Face Hub](https://huggingface.co/models) has many models that have been pre-trained for you to use. One of the models hosted there is a <a href='https://huggingface.co/aychang/roberta-base-imdb'>sentiment classifer that was trained on the entire IMDB corpus</a>. We can load this model and see how it performs on our held-out data.


In [ ]:
tiny_bert = pipeline("text-classification", "arnabdhar/tinybert-imdb")


In [ ]:
# converting to list since thats the input format this model uses
test_list = test_examples['text'].tolist()

# applying the model

results = tiny_bert(inputs =test_list, max_length=512, truncation=True)
# looking at the first five results
results[:5]

In [ ]:
# reformatting to match our original test labels 
tiny_bert_preds = [int(i['label']=="POSITIVE") for i in results]

In [ ]:
print(classification_report(test_labels, tiny_bert_preds, 
                            # add target_names to show labels in the report:
                              target_names=['Negative', 'Positive']))


# add cohen's kappa and balanced accuracy
print("cohens kappa: ", cohen_kappa_score(test_labels, tiny_bert_preds))#
print("balanced accuracy: ", balanced_accuracy_score(test_labels, tiny_bert_preds))

## Other Types of Sentiment

The nice thing about these models is that they are also pre-trained to do different types of sentiment analysis. For example, let's take the Distilbert-base-uncased-emotion model. This provides scores for emotions such as joy or anger. Here's an example of getting the emotions expressed in the first 100 rows the the reviews data set:

In [ ]:
classifier = pipeline("text-classification",
                      model='bhadresh-savani/distilbert-base-uncased-emotion', 
                      top_k=None)


In [ ]:
lst = ["This is an angry sentence", 
       "I'm very afraid",
       "I'm depressed!"
]

prediction = classifier(lst, truncation=True, max_length=120)


In [ ]:
prediction

In [ ]:
emotion_prediction = pd.concat([pd.DataFrame(i) for i in prediction])
emotion_prediction.groupby('label').agg({'score':['min','max','median','mean']})

You can check out some other options on the hugging face <a href='https://huggingface.co/models'>models page.</a>

# BertTopic

[BertTopic](https://maartengr.github.io/BERTopic/api/bertopic.html) is a transformer-based method for topic modeling that works especially well on short texts. You can read more about the algorithm [here](https://maartengr.github.io/BERTopic/algorithm/algorithm.html)

You can try it out with the code below, but be warned that it will take a while to run. For the sake of a speedier model, we'll work with just the headlines and first few sentences from the Fox/CNN news articles from a couple weeks ago. The BertTopic package also provides several built-in functions that make it easy to visualize and explore results, some of which are demonstrated below.

In [ ]:
# Uncomment to install packages:
#%pip install BERTopic
#%pip install SentenceTransformer
#%pip install UMAP

In [ ]:
# import BERTopic (this is very slow sometimes)
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from nltk.tokenize import sent_tokenize


# Loading the 4,000 articles sampled from CNN and Fox
articles = pd.read_csv('https://github.com/Neilblund/APAN/raw/main/news_sample.csv')
# doing some truncation and combining here just to make the model run faster
docs= [' '.join(sent_tokenize(i + " "+ j)[:5]).strip() for i, j in  zip(articles.headline, articles.text)]
# embedding the texts (typically the slowest part)
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=True)
# setting the UMAP model (this makes results reproducible)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=119)
# Train BERTopic
topic_model = BERTopic(umap_model = umap_model).fit(docs, embeddings)

From here, we can try out some of the built-in functions for exploring the results of the topic model:

In [ ]:
# view all topics and the top terms associated with each one
topic_model.topic_labels_

In [ ]:
# view representative documents for topic 0
topic_model.get_representative_docs(0)

In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.head(n=15)

In [ ]:
# heatmap showing textual similarity of each topic 
topic_model.visualize_heatmap()


In [ ]:
# visualize distance between topics
topic_model.visualize_topics()

In [ ]:
# selected topics associated with each outlet (normalized by their overall frequency)
topics_per_class = topic_model.topics_per_class(first_200, articles.source)
topic_model.visualize_topics_per_class(topics_per_class,  topics=[1, 3, 8, 11,12, 13], normalize_frequency=True)

In [ ]:
# documents in space 
topic_model.visualize_documents(articles.headline, embeddings=embeddings)


In [ ]:
# terms for selected topics
topic_model.visualize_barchart([0, 1, 2, 3,11, 10], n_words=10)

In [ ]:
# view topics over time (this model also allows topics themselves to vary slightly depending on the time period)
# see https://maartengr.github.io/BERTopic/getting_started/topicsovertime/topicsovertime.html
topics_over_time = topic_model.topics_over_time(docs, articles.date, nr_bins=15)

In [ ]:
# top 5 topics
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=5)
# or try this to see specific topics:
#topic_model.visualize_topics_over_time(topics_over_time, topics = [0, 1, 2])


In [ ]:
# get topic distributions across individual terms
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs, calculate_tokens=True)



In [ ]:
# plot the topic-word distribution for the first document:
df = topic_model.visualize_approximate_distribution(docs[0], topic_token_distr[0])
df